In [ ]:

from scipy.sparse import hstack

def balance(data):
    accept_data = data[data.binary_state == 1].sample((data.binary_state == 0).sum())
    reject_data = data[data.binary_state == 0]
    data = pd.concat([accept_data, reject_data])
    return data

def data_pipeline(data, use_usd=False, bl=False):
    if bl:
        data = balance(data)

    X = data[['usd_amount', 'country', 'currency', 'cvv_provided']]
    y = data[['binary_state']]

    enc = OneHotEncoder(handle_unknown='ignore', drop='if_binary')
    enc.fit(X[['country', 'currency', 'cvv_provided']])
    encodes = enc.transform(X[['country', 'currency', 'cvv_provided']])
    name = enc.get_feature_names()
    if use_usd:
        usd = np.array(X['usd_amount']).reshape(-1, 1)
        X = hstack((encodes, usd))
        name = np.append(name, 'USD_Amount')
    else:
        X = encodes
    return X, y, name

In [ ]:
X, y, feature_name = data_pipeline(acceptance, use_usd=True, bl=True)
print()
for i in range(1, 50):
    clf = tree.DecisionTreeClassifier(max_depth=i)
    clf.fit(X, y)
    y_pred = clf.predict(X)
    print(f'max depth: {i}, {accuracy_score(y, y_pred)}')

clf = tree.DecisionTreeClassifier(max_depth=10)
clf.fit(X, y)

In [ ]:
import graphviz
dot_data = tree.export_graphviz(clf, out_file=None,
                                feature_names=feature_name,
                                class_names={0:'Reject', 1:'Success'},
                                filled=True, rounded=True,
                                special_characters=True)
graph = graphviz.Source(dot_data)
graph